In [1]:
%matplotlib inline

# To reload external scripts automatically
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import sys
sns.set_context('notebook')

# Importing external files
sys.path.append('scripts/')
from data_import import *
from similarities import *

# 0-Exploratory
This notebook contain the first study of the data to identify what is needed and what we do not need. It will only work with subsets of datasets as they will not fit into memory.

In [2]:
DATA_FOLDER = "../../Project-Data/"
META_FOLDER = DATA_FOLDER + "meta/"
REVIEWS_FOLDER = DATA_FOLDER + "reviews/"
CORE_FOLDER = DATA_FOLDER + "5_core/"
DUMP_FOLDER = DATA_FOLDER + "dump/"
CATEGORIES = ['Books','Movies_and_Tv','Electronics']
MAXCOUNT = -1

## 0.0-Content of the files

The different columns of the **metadata** files are : 
* ```asin``` : the unique identifier of the object
* ```brand```
* ```categories``` : the categories of the object
* ```description``` : the description of the object
* ```imUrl```  : the link toward the images related to the object
* ```price```
* ```related``` : a list of objects that are related to this object
* ```salesRank``` 
* ```title```

The different columns of the **reviews** and **5-core** files are :
* ```asin``` : the unique identifier of the object
* ```helpful``` : a list of 2 integers [x,y], the helpfulness score is x/y votes
* ```overall``` : the rating of the object
* ```reviewText```
* ```reviewTime```
* ```reviewerID```
* ```reviewerName```
* ```summary ``` : the title of the review
* ```unixReviewTime``` : in Unix format

Therefore we keep only the column that are of interest for our task

In [3]:
meta_interesting_cols = ['asin', 'title', 'salesRank', 'description']
review_interesting_cols = ['asin', 'overall', 'unixReviewTime']

## 0.1-Books

In [4]:
meta_books_path, review_books_path, core_book_path = get_paths(0, DATA_FOLDER, META_FOLDER,CORE_FOLDER,
                                               REVIEWS_FOLDER, CATEGORIES)

Paths : 
	 meta = ../../Project-Data/meta/meta_Books.json
	 review = ../../Project-Data/reviews/reviews_Books.json
	 core_path = ../../Project-Data/5_core/Books.json


### 0.1.1-Metadata

In [5]:
meta_books = import_interesting_cols(meta_books_path,DUMP_FOLDER,True,meta_interesting_cols,max_count=MAXCOUNT)
meta_books.head()

Retrieving from : ../../Project-Data/dump/meta_Books_asin_title_salesRank_description_ALL
It took 00:00:03.621 to import the data.


,asin,description,"salesRank_Arts,_Crafts_&_Sewing",salesRank_Books,salesRank_Cell_Phones_&_Accessories,salesRank_Clothing,salesRank_Electronics,salesRank_Health_&_Personal_Care,salesRank_Home_&_Kitchen,salesRank_Industrial_&_Scientific,salesRank_Jewelry,salesRank_Kitchen_&_Dining,salesRank_Movies_&_TV,salesRank_Music,salesRank_Musical_Instruments,salesRank_Office_Products,salesRank_Shoes,salesRank_Sports_&_Outdoors,salesRank_Toys_&_Games,title
0,0001048791,NaN,NaN,6334800.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"The Crucible: Performed by Stuart Pankin, Jero..."
1,0001048775,William Shakespeare is widely regarded as the ...,NaN,13243226.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Measure for Measure: Complete & Unabridged
2,0001048236,"""One thing is certain, Sherlockians, put aside...",NaN,8973864.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,The Sherlock Holmes Audio Collection
3,0000401048,NaN,NaN,6448843.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,The rogue of publishers' row;: Confessions of ...
4,0001019880,NaN,NaN,9589258.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Classic Soul Winner's New Testament Bible


### 0.1.2-Reviews

In [45]:
review_books = import_interesting_cols(
    review_books_path, DUMP_FOLDER,False, review_interesting_cols, max_count=MAXCOUNT)
review_books.head()

Processing the JSON \

KeyboardInterrupt: 

### 0.1.3 - 5-Core

In [6]:
core_books = import_interesting_cols(
    core_book_path,
    DUMP_FOLDER,
    False,
    review_interesting_cols,
    max_count=MAXCOUNT)
core_books.head()

Retrieving from : ../../Project-Data/dump/Books_asin_overall_unixReviewTime_ALL
It took 00:00:00.596 to import the data.


,asin,overall,unixReviewTime
0,000100039X,5.0,2012-12-16
1,000100039X,5.0,2003-12-11
2,000100039X,5.0,2014-01-18
3,000100039X,5.0,2011-09-27
4,000100039X,5.0,2002-10-07


## 0.2-Movies and TV

In [ ]:
meta_movie_path, review_movie_path, core_movie_path = get_paths(
    1, DATA_FOLDER, META_FOLDER, CORE_FOLDER, REVIEWS_FOLDER, CATEGORIES)

### 0.2.1-Metadata

In [ ]:
meta_movie = import_interesting_cols(
    meta_movie_path,DUMP_FOLDER, True, meta_interesting_cols, max_count=MAXCOUNT, dropna=False)
meta_movie.head()

### 0.2.2-Reviews

In [ ]:
review_movie = import_interesting_cols(
    review_movie_path,
    DUMP_FOLDER,
    False,
    review_interesting_cols,
    max_count=100,
    dropna=False)
review_movie.head()

## 0.3-Electronics

In [ ]:
meta_electronic_path, review_electronic_path, core_electronic_path = get_paths(
    2, DATA_FOLDER, CORE_FOLDER, META_FOLDER, REVIEWS_FOLDER, CATEGORIES)

### 0.3.1-Metadata

In [ ]:
meta_electronic = import_interesting_cols(
    meta_electronic_path,
    DUMP_FOLDER,
    True,
    meta_interesting_cols,
    max_count=MAXCOUNT,
    dropna=False)
meta_electronic.head(2)

### 0.3.2-Review

In [ ]:
review_electronic = import_interesting_cols(
    review_electronic_path,
    DUMP_FOLDER,
    False,
    review_interesting_cols,
    max_count=MAXCOUNT,
    dropna=False)
review_electronic.head()

## 1-Detecting similar products
In this section we try to obtain the products that are similar among a given category and to develop a systematic way to detect those similar products.

## 1.1-Books

Some books are sold in different formats : hard cover, pocket, electronic, etc. Therefore in order to compare them we will focus on the title and description that are the only two attributes that should be highly similar between the two different yet similar books.

In [60]:
meta_books = import_interesting_cols(meta_books_path,DUMP_FOLDER,True,meta_interesting_cols,max_count=MAXCOUNT)
core_asin = list(core_books['asin'].unique())

# We delete all the rows where either the title or description is null and keep only core books
core_meta_df = meta_books[meta_books['title'].notnull()]
core_meta_df = core_meta_df[core_meta_df['description'].notnull()]
core_meta_df = core_meta_df[core_meta_df['asin'].isin(core_asin)]
book_desc_titles = core_meta_df[['asin','title','description']]
book_desc_titles = book_desc_titles.set_index(['asin'])
book_desc_titles.head()

Retrieving from : ../../Project-Data/dump/meta_Books_asin_title_salesRank_description_ALL
It took 00:00:05.175 to import the data.


,title,description
asin,,
0001055178,Master Georgie,Beryl Bainbridge seems drawn to disaster. Firs...
000171287X,The Berenstains' B Book (Bright & Early Books),"By Stan Berenstain and Jan Berenstain, Illustr..."
000100039X,The Prophet,"In a distant, timeless place, a mysterious pro..."
0001473905,Rightly Dividing the Word,--This text refers to thePaperbackedition.
0001714538,The Berenstain Bears on the Moon (Bright and E...,PreSchool-Grade 2 A delightful tale told in rh...


In [61]:
book_desc_titles.index.is_unique

True

We first try to experiment with tf-idf weights to find common titles among books. Due to the large size of the database, this task cannot be executed on all the books, we decided to focus as suggested by the dataset on the 5-core dataset. The 5 core dataset contains only products and reviews such that reviewers and products have at least 5-reviews. This will allow us in a first analysis to reduce the number of products that we consider. Since we will need to have at least 5 reviews anyway to look at the **Herding effect**.

### 1.1.0 - TF-IDF

In [48]:
import nltk, string
from sklearn.feature_extraction.text import TfidfVectorizer

#nltk.download('punkt') # if necessary...

stemmer = nltk.stem.porter.PorterStemmer()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)


def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]


def normalize(text):
    '''remove punctuation, lowercase, stem'''
    toReturn = stem_tokens(
        nltk.word_tokenize(text.lower().translate(remove_punctuation_map)))
    return toReturn


vectorizer = TfidfVectorizer(tokenizer=normalize, stop_words='english')


def cosine_sim(text1, text2):
    tfidf = vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0, 1]

### 1.1.1 - Locality sensitive hashing
Based on this [article](http://dataconomy.com/2017/06/locality-sensitive-hashing-pydata/), and following the theory that can be read at the chapter 3.4 of [Mining of massive datasets by Leskovec, Rajaraman and Milliway](http://infolab.stanford.edu/~ullman/mmds/book.pdf) we will try to find similar books.

#### The ```minhash```

We need to detect near similar books in this large datasets. To do so we would need to compare each of the attribute of a given book with all the others which is extremely computationaly expensive. There we will use LSH. Right now the title and description of the book have variable length depending on the books. To perform LSH we would like to have a fixed length representation of the document without modifying the semantics of document similarity.
* first we introduce the principle of shingles. A shingle of 5-gram (hence we discard the last shingle as it only consists in the $<5$ last characters of the document) e.g. is a set of all possible 5-grams in the string. 

In [28]:
document = "Lorem Ipsum dolor sit amet"
shingles = get_shingles(document)

['Lorem',
 'orem ',
 'rem I',
 'em Ip',
 'm Ips',
 ' Ipsu',
 'Ipsum',
 'psum ',
 'sum d',
 'um do',
 'm dol',
 ' dolo',
 'dolor',
 'olor ',
 'lor s',
 'or si',
 'r sit',
 ' sit ',
 'sit a',
 'it am',
 't ame']

In [29]:
other_document = 'Lorem Ipsum dolor sit amet with some extra garbage'
other_shingles = get_shingles(other_document)
jaccard_dist(shingles,other_shingles)

0.4666666666666667

Each document will still have a different number of shingles depending on its length. Therefore we wish to represent it using a *fixed length representation*. We will present now a function ```minhash``` that has a collision probability that is exactly the jaccard similarity (based on this [document](http://infolab.stanford.edu/~ullman/mmds/ch3.pdf) and following this [repo](https://github.com/mattilyra/LSH/blob/master/examples/Introduction.ipynb) . We explain this now :
1. Suppose that the following dataframe is the boolean variable corresponding to whether or not a shingle belongs to a document
2. the ```minhash``` of a document returns a random permutation of the rows and then the first row number it founds where the value is non 0.
    * therefore row 1 for doc 1
    * and row 0 for doc 2

In [32]:
ex_minhash = pd.DataFrame({'shingleID':[1,2,3,4,5,6],'doc1':[0,1,0,0,1,1],'doc2':[1,1,1,0,1,0]})
ex_minhash

,doc1,doc2,shingleID
0,0,1,1
1,1,1,2
2,0,1,3
3,0,0,4
4,1,1,5
5,1,0,6


But this is for this particular permutation, if we had another one like below then the ```minhash``` would return :
   * row 2 for doc1
   * row 1 for doc2

There are a lot of such permutations. Nevertheless we only care about having the same row for each document which mean we have collision of the ```minhash```. Therefore we have only two rows for which this happens : $\text{shingleID} \in \{2,5\}$. Hence the probability that the two doc have the same ```minhash``` (remmember that rows with two zeros do not count as they are not considered by ```minhash```) is the number of rows where both doc value is 1 divided by the number of rows where they are different : $\frac{2}{5}$

In [34]:
ex_minhash.sample(frac=1)

,doc1,doc2,shingleID
2,0,1,3
3,0,0,4
5,1,0,6
1,1,1,2
4,1,1,5
0,0,1,1


We can chose the length of the fingerprint that is returned by the ```minhash```, as we increase the length we get less variance from the random initialisation of the ```minhasher``` but this also greatly increases the memory usage.

#### The principle of LSH

In [54]:
from lsh import minhash
hasher = minhash.MinHasher(seeds=100, char_ngram=5)
fingerprint0 = hasher.fingerprint('Lorem Ipsum dolor sit amet'.encode('utf8'))
len(fingerprint0)

100

Therefore the hash of a document is composed of a given (```seeds```) number of ```minhashes```. We divide this number of ```minhashes``` into a given number of parts (e.g. 5). Since every single ```minhash``` has the jaccard similarity probability of collision then each of the 5 parts will have this probability as well. These parts represent the **locality** in the term LSH.

Then we hash the content of each part using a different hash function to obtain the *binID* that represents the **hashing** part of the method. Into each bin with *binID* we store the entire fingerprint of the document.

The idea is then that we compare documents that fall in the same bins : we will compare their fingerprint which is equivalent to looking at their Jaccard similarity between shingle sets. Since not all documents will fall into the same bins we have reduced the number of potential candidates.

Indeed we call :
* ```seeds``` :  number of ```minihashes``` that compose the fingerprint
* ```bands``` : the number of bins that we want to use

#### Finding candidate duplicates

In [55]:
dump_path = DUMP_FOLDER + "candidate_dup"
candidates_dup = candidate_duplicates(book_desc_titles,dump_path)
print("We have found {} candidate duplicates".format(len(candidates_dup)))

Retrieving from : ../../Project-Data/dump/candidate_dup
We have found 21116 candidate duplicates


In [56]:
# We then create a dictionnary that gives a 
get_candidate = {}
for p1,p2 in list(candidates_dup):
    if(p1 in get_candidate):
        get_candidate[p1].append(p2)
    else:
        get_candidate[p1] = []
        get_candidate[p1].append(p2)


In [79]:
many_similarities = []
for k in get_candidate:
    if(len(get_candidate[k]) > 3):
        many_similarities.append(k)

In [83]:
def get_titles(asin,get_candidate):
    title_desc_list = []
    for sim in get_candidate[asin]:
        entry = book_desc_titles.loc[sim]
        title = entry['title']
        title_desc_list.append(title)
    return title_desc_list

In [86]:
get_titles(many_similarities[10],get_candidate)

['Russian, Conversational: Learn to Speak and Understand Russian with Pimsleur Language Programs',
 'Dutch, Basic: Learn to Speak and Understand Dutch with Pimsleur Language Programs',
 'German, Conversational: Learn to Speak and Understand German with Pimsleur Language Programs',
 'Thai, Conversational: Learn to Speak and Understand Thai with Pimsleur Language Programs',
 'Greek (Modern), Basic: Learn to Speak and Understand Modern Greek with Pimsleur Language Programs',
 'Hebrew, Basic: Learn to Speak and Understand Hebrew with Pimsleur Language Programs',
 'Arabic (Eastern), Basic: Learn to Speak and Understand Eastern Arabic with Pimsleur Language Programs',
 'Czech, Basic: Learn to Speak and Understand Czech with Pimsleur Language Programs',
 'Turkish, Conversational: Learn to Speak and Understand Turkish with Pimsleur Language Programs',
 'Portuguese (Brazilian), Conversational: Learn to Speak and Understand Brazilian Portuguese with Pimsleur Language Programs']

We can see that we are already doing quite well to detect candidate similars but indeed those are not the same and we now need to go further and define a suite of tools that we can use to check between those candidates.

### 1.1.2 - TF-IDF 

In [42]:
# We pick a random title 
book_1 = core_meta_df.sample(n=1)[['title','description']]
book_1

,title,description
976231,Died He For Me: A Physicians Look at the Cruci...,The death of Jesus for our sins is the heart o...


In [45]:
sub_sample = core_meta_df[['title','description']]
sub_sample['title_similarity'] = sub_sample['title'].map(lambda x: cosine_sim(x,book_1.iloc[0,0]))
sub_sample['description_similarity'] = sub_sample['description'].map(lambda x: cosine_sim(x,book_1.iloc[0,1]))
sub_sample.sort_values(['title_similarity'],ascending=False,inplace=True)
sub_sample.head()

/Users/hmoreau/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


KeyboardInterrupt: 

In [ ]:
sub_sample.sort_values(['description_similarity'],ascending=False,inplace=True)
sub_sample.head()